## **Démo** : **Smart Traffic**

---

### **Required main libraries installations**

In [60]:
!python --version

Python 3.11.0


In [61]:
!python -m pip install --upgrade pip
!pip3 install ultralyticsplus
!python -m pip install --upgrade ultralyticsplus
!pip3 install ultralytics

---

### **YOLOv8 model load**

#### Chargement du modèle YOLO

In [62]:
from ultralyticsplus import YOLO, render_result

# load model
model = YOLO('best.pt')

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

#### Attributs et paramètres important du modèle YOLO

In [63]:
print("Les attributs de l'objet model : \n", list(model.__dict__.keys()))
print("Les attributs secondaires de l'objet model : \n", list(model.model.__dict__.keys()))

Les attributs de l'objet model : 
 ['type', 'ModelClass', 'TrainerClass', 'ValidatorClass', 'PredictorClass', 'predictor', 'trainer', 'task', 'ckpt', 'cfg', 'ckpt_path', 'overrides', 'training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_pre_hooks', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_hooks_always_called', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_state_dict_hooks', '_state_dict_pre_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'callbacks', 'metrics', 'session', 'model_name']
Les attributs secondaires de l'objet model : 
 ['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_pre_hooks', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_hooks_always_called', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_state_dict_hooks', '_state_dict_pre_hooks', '_load_

In [64]:
print("Les noms des classes :")
for i in range(len(model.model.names)):
    print(i, model.model.names[i])

Les noms des classes :
0 big bus
1 big truck
2 bus-l-
3 bus-s-
4 car
5 mid truck
6 small bus
7 small truck
8 truck-l-
9 truck-m-
10 truck-s-
11 truck-xl-


---

### **Vehicle detection and counting from video**

##### ↓ Imports ↓

In [65]:
import cv2
import numpy as np

#### Analyse de véhicule sur la vidéo

In [68]:
# check the path to the vid.avi file
cap = cv2.VideoCapture('media/in/feu4.avi')

# get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# create video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('media/out/output.avi', fourcc, fps, (width, height))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # perform inference
        results = model.predict(frame)
                
        # observe results
        render = render_result(model=model, image=frame, result=results[0])

        # Convert PIL Image to numpy array
        frame_out = np.array(render)

        # save frame to video
        out.write(frame_out)

        # Sortie si 'q' est pressé
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# release resources
cap.release()
out.release()

# avi to mp4
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

cv2.destroyAllWindows()



0: 480x640 16 cars, 137.1ms
Speed: 7.5ms preprocess, 137.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 23 cars, 76.3ms
Speed: 3.0ms preprocess, 76.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 24 cars, 75.2ms
Speed: 2.0ms preprocess, 75.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 24 cars, 75.9ms
Speed: 3.0ms preprocess, 75.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 big bus, 24 cars, 78.7ms
Speed: 2.5ms preprocess, 78.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 big bus, 20 cars, 76.0ms
Speed: 3.0ms preprocess, 76.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 big bus, 23 cars, 76.3ms
Speed: 2.0ms preprocess, 76.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 25 cars, 135.6ms
Speed: 3.0ms preprocess, 135.6ms inference, 1.0ms postprocess

#### Tri et comptage des véhicules détectés

In [69]:
# Initialiser un dictionnaire pour compter les détections pour chaque classe
detection_counts = {}

detections = results[0].boxes.data

for d in detections:
    class_id = int(d[5])
    class_name = model.model.names[class_id]

    if class_name in detection_counts:
        detection_counts[class_name]+=1
    else: detection_counts[class_name]=1

print(f"Dictionnaire de détections: \n", detection_counts)


Dictionnaire de détections: 
 {'car': 14, 'truck-s-': 1}


In [70]:
trucks = (
    detection_counts.get("big truck", 0) +
    detection_counts.get("mid truck", 0) +
    detection_counts.get("small truck", 0) +
    detection_counts.get("truck-l-", 0) +
    detection_counts.get("truck-m-", 0) +
    detection_counts.get("truck-s-", 0) +
    detection_counts.get("truck-xl-", 0)
)

buses = (
    detection_counts.get("bus-s-", 0) +
    detection_counts.get("big bus", 0) +
    detection_counts.get("small bus", 0) +
    detection_counts.get("bus-l-", 0)
)

cars = detection_counts.get("car", 0)

print(f"Nombre de camions: {trucks}\n\nNombre de bus: {buses}\n\nNombre de voitures: {cars}")

Nombre de camions: 1

Nombre de bus: 0

Nombre de voitures: 14


---

### **Connect to a SQLite database**

##### ↓ Imports ↓

In [71]:
import sqlite3
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

###### Chemin d'accès vers la base de données

In [72]:
import os
db_path = os.path.join(os.getcwd(), "db-copy.sqlite3")
print(db_path)


c:\Users\chari\OneDrive\Desktop\KESKIA\MontoringSmartTraffic\project\yolo_project\src\db-copy.sqlite3


#### Réinitialiser les données de base

In [78]:
import sqlite3

def reinitialiser_donnees():
    # Connect to the SQLite database
    conn = sqlite3.connect('db-copy.sqlite3')
    c = conn.cursor()

    # Temporarily disable foreign key constraints
    c.execute("PRAGMA foreign_keys = OFF;")

    # Delete all data from the tables in the correct order
    tables = ['app_detectionvehicule', 'app_detection', 'app_vehicule', 'app_feu', 'app_etatfeu', 'app_rue', 'app_typevehicule', 'app_temps']
    for table in tables:
        c.execute(f"DELETE FROM {table};")

    # Re-enable foreign key constraints
    c.execute("PRAGMA foreign_keys = ON;")

    # Insert data into the tables
    c.execute("INSERT INTO app_rue (idRue ,nomRue) VALUES (1, 'Route d''Orléans'), (2, 'Route de Massy');")
    c.execute("INSERT INTO app_typevehicule (idTypeVehicule, nomTypeVehicule) VALUES (1, 'Voiture'), (2, 'Camion'), (3, 'Bus');")
    c.execute("INSERT INTO app_etatfeu (idEtatFeu, nomEtatFeu) VALUES (1, 'Vert'), (2, 'Rouge'), (3, 'Orange'), (4, 'Eteint');")
    c.execute("INSERT INTO app_feu (idFeu, nomFeu, rue_id, etatFeu_id) VALUES (1, 'Feu1', 1, 1), (2, 'Feu2', 1, 1), (3, 'Feu3', 2, 2), (4, 'Feu4', 2, 2);")

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

reinitialiser_donnees()

#### Insertion des détections sur la base SQLite

In [79]:
import sqlite3
import datetime

# Fonction pour insérer les détections
def inserer_detections(id_type_vehicule, nombre_detections, id_rue, id_feu, curseur, dateH, semaine, jour, weekEnd):
    for i in range(nombre_detections):
        curseur.execute("INSERT INTO app_temps (dateHeure, semaine, jourSemaine, isWeekEnd) VALUES (?, ?, ?, ?)", (dateH, semaine, jour, weekEnd))    
        idTemps = curseur.lastrowid

        curseur.execute("INSERT INTO app_detection (temps_id, rue_id, feu_id) VALUES (?, ?, ?)", (idTemps, id_rue, id_feu))
        idDetection = curseur.lastrowid

        curseur.execute("INSERT INTO app_vehicule (nomVehicule, typevehicule_id) VALUES (?, ?)", ("Véhicule", id_type_vehicule))
        idVehicule = curseur.lastrowid

        curseur.execute("INSERT INTO app_detectionvehicule (vehicule_id, detection_id) VALUES (?, ?)", (idVehicule, idDetection))

# Fonction pour réinitialiser le compteur d'auto-incrémentation des ids
def reset_autoincrement(cursor, table_name):
    cursor.execute(f"UPDATE sqlite_sequence SET seq = 0 WHERE name = '{table_name}'")

# Récupération des données temporelles
dateH = datetime.datetime.now()
semaine = dateH.isocalendar()[1]
jour = dateH.strftime("%A").capitalize()
weekEnd = jour in ["Samedi", "Dimanche"]

def insertion(id_rue, id_feu):
    # Connexion à la base de données avec un gestionnaire de contexte
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            isDetection = False
            reset_autoincrement(curseur, 'app_temps')
            reset_autoincrement(curseur, 'app_detection')
            reset_autoincrement(curseur, 'app_vehicule')
            reset_autoincrement(curseur, 'app_detectionvehicule')

            if buses > 0:
                isDetection = True
                inserer_detections(3, buses, id_rue, id_feu, curseur, dateH, semaine, jour, weekEnd)


            if cars > 0:
                isDetection = True
                inserer_detections(1, cars, id_rue, id_feu, curseur, dateH, semaine, jour, weekEnd)


            if trucks > 0:
                isDetection = True
                inserer_detections(2, trucks, id_rue, id_feu, curseur, dateH, semaine, jour, weekEnd)

            if not isDetection:
                print("Pas de véhicule détecté !")


            # La transaction est automatiquement validée (commit) en sortant du bloc with
    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)

id_rue, id_feu = 1, 3
insertion(id_rue, id_feu)


In [80]:
def afficher_all_detections():
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            detections = curseur.execute("SELECT * FROM app_detection")
            for detection in detections:
                print(detection)
    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)
afficher_all_detections()

(1, 1, 1, 3)
(2, 1, 2, 3)
(3, 1, 3, 3)
(4, 1, 4, 3)
(5, 1, 5, 3)
(6, 1, 6, 3)
(7, 1, 7, 3)
(8, 1, 8, 3)
(9, 1, 9, 3)
(10, 1, 10, 3)
(11, 1, 11, 3)
(12, 1, 12, 3)
(13, 1, 13, 3)
(14, 1, 14, 3)
(15, 1, 15, 3)


#### Visionnage des données dans la base

In [ ]:
import os

db_path = os.path.join(os.getcwd(), "db-copy.sqlite3")
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file not found: {db_path}")

##### Les véhicules

In [ ]:
def afficher_all_vehicules():
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            detections = curseur.execute("SELECT * FROM app_vehicule")
            for detection in detections:
                print(detection)
    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)
afficher_all_vehicules()

##### Les véhicules par feux

In [ ]:
def afficher_vehicules_by_feux():
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            detections = curseur.execute(f"SELECT count(idDetection), feu_id FROM app_detection GROUP BY feu_id")
            for detection in detections:
                print(detection)
    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)

def afficher_vehicules_by_feu(feu):
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            detections = curseur.execute(f"SELECT count(idDetection), feu_id FROM app_detection WHERE feu_id = {feu} GROUP BY feu_id")
            for detection in detections:
                print(detection)
    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)

afficher_vehicules_by_feux()
afficher_vehicules_by_feu(3)
            

##### Les véhicules regroupés par type

In [ ]:
def afficher_vehicules_by_types():
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            detections = curseur.execute("SELECT count(id), nomTypeVehicule FROM app_detectionvehicule INNER JOIN app_vehicule ON vehicule_id=idVehicule INNER JOIN app_typevehicule ON typevehicule_id=idTypeVehicule GROUP BY typevehicule_id")
            for detection in detections:
                print(detection)
    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)

# def afficher_vehicules_by_type(nomType):
#     try:
#         with sqlite3.connect("db-copy.sqlite3") as connexion:
#             curseur = connexion.cursor()
#             detections = curseur.execute(f"SELECT count(id), nomTypeVehicule FROM app_detectionvehicule INNER JOIN app_vehicule ON vehicule_id=idVehicule INNER JOIN app_typevehicule ON typevehicule_id=idTypeVehicule WHERE app_typevehicule.nomTypeVehicule = {nomType} GROUP BY typevehicule_id")
#             for detection in detections:
#                 print(detection)
#     except sqlite3.Error as e:
#         print("Une erreur est survenue lors de la connexion à la base de données:", e)*

afficher_vehicules_by_types()
# afficher_vehicules_by_type("Bus")

#### Graphiques des données

##### Graphique n°1 : véhicules détectés par feu

In [ ]:
import matplotlib.pyplot as plt
import sqlite3

def graphique_vehicule_by_feu(feu_id):
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            detections = list(curseur.execute(f"SELECT count(idDetection), feu_id FROM app_detection WHERE feu_id = {feu_id} GROUP BY feu_id"))
            data = [d[0] for d in detections] 
            feu_ids = [d[1] for d in detections]  

            # create the bar chart
            plt.bar(feu_ids, data)
            plt.xticks(feu_ids)
            plt.yticks(data)
            plt.xlabel('Feu')
            plt.ylabel('Nombre de véhicules')
            plt.title('Véhicules détectés par feu')
            plt.show()

    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)

graphique_vehicule_by_feu(3)

##### Graphique n°2 : véhicules détectés par type

In [ ]:
import matplotlib.pyplot as plt
import sqlite3

def graphique_vehicule_by_type():
    try:
        with sqlite3.connect("db-copy.sqlite3") as connexion:
            curseur = connexion.cursor()
            detections = list(curseur.execute("SELECT count(id), nomTypeVehicule FROM app_detectionvehicule INNER JOIN app_vehicule ON vehicule_id=idVehicule INNER JOIN app_typevehicule ON typevehicule_id=idTypeVehicule GROUP BY typevehicule_id"))
            data = [d[0] for d in detections] 
            type_vehicules = [d[1] for d in detections]  

            # create the bar chart
            plt.bar(type_vehicules, data)
            plt.xticks(type_vehicules)
            plt.yticks(data)
            plt.xlabel('Type de véhicule')
            plt.ylabel('Nombre de véhicules')
            plt.title('Véhicules détectés par type')
            plt.show()

    except sqlite3.Error as e:
        print("Une erreur est survenue lors de la connexion à la base de données:", e)

graphique_vehicule_by_type()


### &copy; 2024 Smart Traffic
